# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,12.63,72,79,5.11,NZ,1673456986
1,1,cape town,-33.9258,18.4232,24.50,67,96,5.36,ZA,1673456703
2,2,ushuaia,-54.8000,-68.3000,10.81,71,75,7.72,AR,1673456986
3,3,norfolk,42.1668,-71.2495,2.54,68,40,4.12,US,1673456987
4,4,grand gaube,-20.0064,57.6608,26.32,72,67,2.24,MU,1673456987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,hilo,19.7297,-155.0900,21.82,56,0,2.57,US,1673457023
100,100,cabo san lucas,22.8909,-109.9124,25.28,60,0,3.09,MX,1673456962
133,133,pandharpur,17.6667,75.3333,23.19,20,0,2.15,IN,1673457076
341,341,salalah,17.0151,54.0924,24.05,64,0,1.03,OM,1673457177
380,380,kahului,20.8947,-156.4700,23.80,75,0,2.68,US,1673457197
394,394,makakilo city,21.3469,-158.0858,23.18,88,0,2.06,US,1673457204
423,423,richards bay,-28.7830,32.0377,23.30,77,0,4.17,ZA,1673457216
500,500,alacranes,22.7675,-81.5675,26.27,43,0,4.12,CU,1673457250
507,507,ratnagiri,16.9833,73.3000,22.09,64,0,2.74,IN,1673457254
521,521,nanakuli,21.3906,-158.1547,23.97,88,0,2.06,US,1673457262


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,hilo,US,19.7297,-155.0900,56,
100,cabo san lucas,MX,22.8909,-109.9124,60,
133,pandharpur,IN,17.6667,75.3333,20,
341,salalah,OM,17.0151,54.0924,64,
380,kahului,US,20.8947,-156.4700,75,
394,makakilo city,US,21.3469,-158.0858,88,
423,richards bay,ZA,-28.7830,32.0377,77,
500,alacranes,CU,22.7675,-81.5675,43,
507,ratnagiri,IN,16.9833,73.3000,64,
521,nanakuli,US,21.3906,-158.1547,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
pandharpur - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
alacranes - nearest hotel: No hotel found
ratnagiri - nearest hotel: No hotel found
nanakuli - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
18,hilo,US,19.7297,-155.0900,56,No hotel found
100,cabo san lucas,MX,22.8909,-109.9124,60,No hotel found
133,pandharpur,IN,17.6667,75.3333,20,No hotel found
341,salalah,OM,17.0151,54.0924,64,No hotel found
380,kahului,US,20.8947,-156.4700,75,No hotel found
394,makakilo city,US,21.3469,-158.0858,88,No hotel found
423,richards bay,ZA,-28.7830,32.0377,77,No hotel found
500,alacranes,CU,22.7675,-81.5675,43,No hotel found
507,ratnagiri,IN,16.9833,73.3000,64,No hotel found
521,nanakuli,US,21.3906,-158.1547,88,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)